# Use SMPC Presentation databases to produce Program entries

In [1]:
import collections
import os
from os import listdir, chdir,makedirs
import shutil
import io  
import sys  
import urllib
import csv
from pylatexenc.latexencode import utf8tolatex
from pylatexenc.latex2text import LatexNodes2Text

from time import strptime
from datetime import datetime
from datetime import date

from dumptruck import DumpTruck
from collections import OrderedDict
from collections import Counter
from collections import deque
from os import listdir, chdir,makedirs

# Define functions for LaTeX formating of presentations in Program

In [2]:
# aweful list of specific conversions for characters that freak out ascii and LaTeX for these entries, sans abstracts
def cleanLaTeX(textstring):
    a = utf8tolatex(textstring)
    a = a.replace("ɔ".decode('utf-8'), "\\textipa{O}")
    a = a.replace("ɛ".decode('utf-8'), "\\textipa{3}")
    a = a.replace("x̄".decode('utf-8'), "$\\bar{x}$")
    a = a.replace("↔".decode('utf-8'), "$\leftrightarrow$")
    a = a.replace("♭".decode('utf-8'), "$\\flat$")
    a = a.replace("| אוניברסיטת תל-אביב".decode('utf-8'), "")
    a = a.replace("ṭ".decode('utf-8'), "\d{t}").encode('utf-8')   
    #a = cleanLaTeX(textstring)
    return a

In [3]:
# function to index affiliations per submission, 
# input list with entry per author
# output unique list of affiliations and matching index per author
#
# note: this only allows for single affilitions per author. 
#       too much work to redefine combos of institutions and collab centres per author
def InstSort(InstitutionList):
    #find unique Institutions
    InstList = []
    for inst in InstitutionList:
        yesAdd = 1
        for est in InstList:
            if inst.strip() == est:
                yesAdd = 0
        if yesAdd==1:
            InstList.append(inst.strip())
    InstIndex = []
    
    for inst in InstitutionList:
        for k in range(len(InstList)):
            if InstList[k] == inst.strip():
                InstIndex.append(k+1)
    
    InstitutionsSorted = [InstIndex, InstList]
    return InstitutionsSorted

In [4]:
# function producing latex code for a talk entry. 
# Note this code sits within \begin{tabulature} 
# Also, quirky extra slashes around to circumvent string interpretation via f.write

def talkDets(talk):
    TalkCode = ("%s %s-%i" % (talk['Time'],talk['SessionCode'],talk['TalkSessionNumber']))
    SS = ("\\textit{%s} & \\textbf{%s}\\\\\n" % (TalkCode,cleanLaTeX(talk['TalkTitle'])))

    Is = InstSort(talk['Institutions'])
    InstIndex = Is[0]
    InstList = Is[1]
    
    authors = '& {\small '
    for i in range(len(talk['Authors'])):
        authors += cleanLaTeX(talk['Authors'][i].strip('*')) # Author
        authors += '$^' + str(InstIndex[i]) + '$'# Institution affiliations
        authors += '\index{'+ cleanLaTeX(talk['AuthorIndex'][i]) +'}' # index
        if i < len(talk['Authors'])-1:
            authors += ', '
    authors += '}\\\\\n'
    
    instit =  '& {\scriptsize {\itshape{$^1$' + cleanLaTeX(Is[1][0])
    for i in range(len(Is[1])-1):
        instit += ', $^' + str(i+2) + '$' + cleanLaTeX(Is[1][i+1])
    instit += '}}}\\\\\n'

    texTalk = SS + authors + instit
    
    return texTalk

In [5]:
# function producing latex code for a talk entry that is part of a symposium, eg, no talk time 
# Note this code sits within \begin{tabulature} 

def SympDets(talk):
    TalkCode = ("%s-%i" % (talk['SessionCode'],talk['TalkSessionNumber']))
    SS = ("\\textit{%s} & \\textbf{%s}\\\\\n" % (TalkCode,cleanLaTeX(talk['TalkTitle'])))
    # Accept Symposium
    Is = InstSort(talk['Institutions'])
    InstIndex = Is[0]
    InstList = Is[1]
    
    authors = '& {\small '
    for i in range(len(talk['Authors'])):
        authors += cleanLaTeX(talk['Authors'][i].strip('*')) # Author
        authors += '$^' + str(InstIndex[i]) + '$'# Institution affiliations
        authors += '\index{'+ cleanLaTeX(talk['AuthorIndex'][i]) +'}' # index
        if i < len(talk['Authors'])-1:
            authors += ', '
    authors += '}\\\\\n'
    
    instit =  '& {\scriptsize {\itshape{$^1$' + cleanLaTeX(Is[1][0])
    for i in range(len(Is[1])-1):
        instit += ', $^' + str(i+2) + '$' + cleanLaTeX(Is[1][i+1])
    instit += '}}}\\\\\n'

    texTalk = SS + authors + instit
    
    return texTalk

In [6]:
# function producing latex code for a poster entry 
# Note this code sits within \begin{tabular} 

def posterDets(poster):
    PosterCode = ("%s-%i" % (poster['PosterSession'],poster['PositionNumber']))
    SS = ("\\textit{%s} & \\textbf{%s}\\\\\n" % (PosterCode,cleanLaTeX(poster['PosterTitle'])))

    Is = InstSort(poster['Institutions'])
    InstIndex = Is[0]
    InstList = Is[1]
    
    authors = '& {\small '
    for i in range(len(poster['Authors'])):
        authors += cleanLaTeX(poster['Authors'][i].strip('*')) # Author
        authors += '$^' + str(InstIndex[i]) + '$'# Institution affiliations
        authors += '\index{'+ cleanLaTeX(poster['AuthorIndex'][i]) +'}' # index
        if i < len(poster['Authors'])-1:
            authors += ', '
    authors += '}\\\\\n'
    
    instit =  '& {\scriptsize \\textit{$^1$' + cleanLaTeX(Is[1][0])
    for i in range(len(Is[1])-1):
        instit += ', $^' + str(i+2) + '$' + cleanLaTeX(Is[1][i+1])
    instit += '}}\\\\\n'

    texPoster = SS + authors + instit #+ abstract + cont + subjects
    
    return texPoster

# Compile the latex files for talks and posters, day by day

In [7]:
chdir('./data')
dt = DumpTruck(dbname='Aug5_Talks.db')
ds = DumpTruck(dbname='Aug5_Sessions.db')
data = ds.dump()
chdir('..')
chdir('./Program')
# OK now write to file
fileName = "Aug5_Talks_Details.tex"
f= open(fileName, 'w+')

f.write("\section*{}\\addcontentsline{toc}{section}{August 5th Presentations}\n\n")
f.write('\\textbf{\Huge August 5th Talks}\n\n \\noindent')
for i in range(len(data)):
    ss = ds.execute('SELECT * FROM `dumptruck` WHERE `SessionNumber` = ' + str(i+1))
    f.write('\n\\vspace{10mm}\n')
    f.write('\\begin{tabular}{AB}\n')
    sessionTitle = ('\\textbf{\Large %s} & \\textbf{\Large %s}\\\\\n \\textbf{%s} & %s \\\\\n  ' % 
                    (ss[0]['SessionCode'], cleanLaTeX(ss[0]['SessionTitle']), ss[0]['Room'],ss[0]['Times']))
    f.write(sessionTitle)
    
    ts = dt.execute('SELECT * FROM `dumptruck` WHERE `SessionNumber` = ' + str(i+1))    
    for talk in ts:
        # print (str(talk['SubmissionNumber']) + ':' + talk['TalkTitle'])
        if talk['Status'] == 'Accept Spoken':
            r = talkDets(talk)
        else:
            r = SympDets(talk)
        f.write(r)
        #print r
    f.write('\n\end{tabular}')
f.close() 
chdir('..')

In [8]:
chdir('./data')
dt = DumpTruck(dbname='Aug6_Talks.db')
ds = DumpTruck(dbname='Aug6_Sessions.db')
data = ds.dump()
chdir('..')
chdir('./Program')

# OK now write to file
fileName = "Aug6_Talks_Details.tex"
f= open(fileName, 'w+')

f.write("\section*{}\\addcontentsline{toc}{section}{August 6th Presentations}\n")
f.write('\\textbf{\Huge August 6th Talks}\n\n \\noindent')
for i in range(len(data)):
    ss = ds.execute('SELECT * FROM `dumptruck` WHERE `SessionNumber` = ' + str(i+1))
    #f.write('\\begin{table}[h]\n')
    f.write('\n\\vspace{10mm}\n')
    f.write('\\begin{tabular}{AB}\n')
    sessionTitle = ('\\textbf{\Large %s} & \\textbf{\Large %s}\\\\\n \\textbf{%s} & %s \\\\\n  ' % 
                    (ss[0]['SessionCode'], cleanLaTeX(ss[0]['SessionTitle']), ss[0]['Room'],ss[0]['Times']))
    f.write(sessionTitle)
    
    ts = dt.execute('SELECT * FROM `dumptruck` WHERE `SessionNumber` = ' + str(i+1))    
    for talk in ts:
        #print (str(talk['SubmissionNumber']) + ':' + talk['TalkTitle'])
        if talk['Status'] == 'Accept Spoken':
            r = talkDets(talk)
        else:
            r = SympDets(talk)
        f.write(r)
        #print r
    f.write('\n\end{tabular}')
    #f.write('\end{table}\n\n')
f.close() 
chdir('..')

In [9]:
chdir('./data')
dt = DumpTruck(dbname='Aug7_Talks.db')
ds = DumpTruck(dbname='Aug7_Sessions.db')
data = ds.dump()
chdir('..')
chdir('./Program')

# OK now write to file
fileName = "Aug7_Talks_Details.tex"
f= open(fileName, 'w+')

f.write('\section*{}\\addcontentsline{toc}{section}{August 7th Presentations}\n\n')
f.write('\\textbf{\Huge August 7th Talks}\n\n \\noindent')
for i in range(len(data)):
    ss = ds.execute('SELECT * FROM `dumptruck` WHERE `SessionNumber` = ' + str(i+1))
    #f.write('\\begin{table}[h]\n')
    f.write('\n\\vspace{10mm}\n')
    f.write('\\begin{tabular}{AB}\n')
    sessionTitle = ('\\textbf{\Large %s} & \\textbf{\Large %s}\\\\\n \\textbf{%s} & %s \\\\\n  ' % 
                    (ss[0]['SessionCode'], cleanLaTeX(ss[0]['SessionTitle']), ss[0]['Room'],ss[0]['Times']))
    f.write(sessionTitle)
    
    ts = dt.execute('SELECT * FROM `dumptruck` WHERE `SessionNumber` = ' + str(i+1))    
    for talk in ts:
        # print (str(talk['SubmissionNumber']) + ':' + talk['TalkTitle'])
        if talk['Status'] == 'Accept Spoken':
            r = talkDets(talk)
        else:
            r = SympDets(talk)
        f.write(r)
        #print r
    f.write('\n\end{tabular}')
    #f.write('\end{table}\n\n')
f.close() 
chdir('..')

In [10]:
chdir('./data')
dt = DumpTruck(dbname='Aug6_Posters.db')
data = dt.dump()
chdir('..')
chdir('./Program')

# OK now write to file
fileName = "Aug6_Poster_Details.tex"
f= open(fileName, 'w+')
f.write("\section*{}\\addcontentsline{toc}{section}{August 6th Posters}\n\n")
f.write('\\textbf{\Huge August 6th Posters}\n\n \\noindent')

i = '1'
ps = dt.execute('SELECT * FROM `dumptruck` WHERE `PosterSessionNumber` = ' + i)
f.write('\n\\begin{table}[H]\n')    
sessionTitle = ('\caption{\\textbf{\Large Poster Session %s, %s}}\n' % 
                (ps[0]['PosterSession'], ps[0]['Time']))
f.write(sessionTitle)
f.write('\\begin{tabular}{DE}\n')   
for poster in ps:
#         print(str(poster['SubmissionNumber']) + ':' + poster['PosterTitle'])
    r = posterDets(poster)
    if (poster['PositionNumber']+5)% 30 < 2:
        f.write('\n\end{tabular}\n \end{table}\n\n')
        f.write('\\begin{table}[H]\n')
        sessionTitle = ('\caption{\\textbf{\Large Poster Session %s, %s, continued }}\n' %  (poster['PosterSession'], poster['Time']))
        f.write(sessionTitle)
        f.write('\\begin{tabular}{DE}\n')  
    f.write(r)
f.write('\n\end{tabular}\n \end{table}')

i = '2'
ps = dt.execute('SELECT * FROM `dumptruck` WHERE `PosterSessionNumber` = ' + i)
f.write('\n\\begin{table}[H]\n')    
sessionTitle = ('\caption{\\textbf{\Large Poster Session %s, %s}}\n' % 
                (ps[0]['PosterSession'], ps[0]['Time']))
f.write(sessionTitle)
f.write('\\begin{tabular}{DE}\n')   
for poster in ps:
#         print(str(poster['SubmissionNumber']) + ':' + poster['PosterTitle'])
    r = posterDets(poster)
    if (poster['PositionNumber'])% 30 < 2:
        f.write('\n\end{tabular}\n \end{table}\n\n')
        f.write('\\begin{table}[H]\n')
        sessionTitle = ('\caption{\\textbf{\Large Poster Session %s, %s, continued }}\n' %  (poster['PosterSession'], poster['Time']))
        f.write(sessionTitle)
        f.write('\\begin{tabular}{DE}\n')  
    f.write(r)
f.write('\n\end{tabular}\n \end{table}')
    
f.close()
chdir('..')

In [11]:
chdir('./data')
dt = DumpTruck(dbname='Aug7_Posters.db')
data = dt.dump()
chdir('..')
chdir('./Program')

# OK now write to file
fileName = "Aug7_Poster_Details.tex"
f= open(fileName, 'w+')

f.write("\section*{}\\addcontentsline{toc}{section}{August 7th Posters}\n\n")
f.write('\\textbf{\Huge August 7th Posters}\n\n \\noindent')

i = '3'
ps = dt.execute('SELECT * FROM `dumptruck` WHERE `PosterSessionNumber` = ' + i)
f.write('\n\\begin{table}[H]\n')    
sessionTitle = ('\caption{\\textbf{\Large Poster Session %s, %s}}\n' % 
                (ps[0]['PosterSession'], ps[0]['Time']))
f.write(sessionTitle)
f.write('\\begin{tabular}{DE}\n')   
for poster in ps:
#         print(str(poster['SubmissionNumber']) + ':' + poster['PosterTitle'])
    r = posterDets(poster)
    if (poster['PositionNumber']+3)% 30 < 2:
        f.write('\n\end{tabular}\n \end{table}\n\n')
        f.write('\\begin{table}[H]\n')
        sessionTitle = ('\caption{\\textbf{\Large Poster Session %s, %s, continued }}\n' %  (poster['PosterSession'], poster['Time']))
        f.write(sessionTitle)
        f.write('\\begin{tabular}{DE}\n')  
    f.write(r)
f.write('\n\end{tabular}\n \end{table}')

i = '4'
ps = dt.execute('SELECT * FROM `dumptruck` WHERE `PosterSessionNumber` = ' + i)
f.write('\n\\begin{table}[H]\n')    
sessionTitle = ('\caption{\\textbf{\Large Poster Session %s, %s}}\n' % 
                (ps[0]['PosterSession'], ps[0]['Time']))
f.write(sessionTitle)
f.write('\\begin{tabular}{DE}\n')   
for poster in ps:
#         print(str(poster['SubmissionNumber']) + ':' + poster['PosterTitle'])
    r = posterDets(poster)
    if (poster['PositionNumber'])% 30 < 2:
        f.write('\n\end{tabular}\n \end{table}\n\n')
        f.write('\\begin{table}[H]\n')
        sessionTitle = ('\caption{\\textbf{\Large Poster Session %s, %s, continued }}\n' %  (poster['PosterSession'], poster['Time']))
        f.write(sessionTitle)
        f.write('\\begin{tabular}{DE}\n')  
    f.write(r)
f.write('\n\end{tabular}\n \end{table}')
 
f.close()
chdir('..')